Hidden channels don’t have a predefined meaning, and it’s up to the update rule to decide what to use them for. They can be interpreted as concentrations of some chemicals, electric potentials or some other signaling mechanism that are used by cells to orchestrate the growth. In terms of our biological analogy - all our cells share the same genome.

(Maybe they can somehow represent the complex state)

In [173]:
from CaAttributes import CaNeighbourhoods, MemoryTypes, RuleTypes
%load_ext autoreload
%autoreload 2
from CaMemory import CaMemory
from ca_funcs import make_glider, make_game_of_life
import tensorflow as tf
import numpy as np
import random
from train_ca import *
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch, logs=None):
        if logs.get('val_accuracy') == 1:
            self.model.stop_training = True 
#Seedings and Config
 #keras.saving.get_custom_objects().clear()
#Seedings and Config
SEED =2
print("start")
os.environ['PYTHONHASHSEED']=str(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # TF 2.1
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.config.threading.set_inter_op_parallelism_threads(1)
# 2100 outer totalistic  Generating Data
samples=2100
data_size, wspan, hspan = (samples, 10, 10)
x_values = np.random.choice([0, 1], (data_size, wspan, hspan), p=[.5, .5])

MEMORY_CONSTANT=2
num_classes = 2  
sequence_length=MEMORY_CONSTANT*2
gol_m = CaMemory(grid_size=10 , rule_type=RuleTypes.OuterTotalistic,
                neighbourhood_type=CaNeighbourhoods.Von_Neumann, memory_type=MemoryTypes.Most_Frequent, memory_horizon=MEMORY_CONSTANT)


gol_m.set_rule([[0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0, 0]])


sequences= np.array(gol_m.generate_training_data_sequences(x_values,sequence_length=sequence_length))
print(f"Training Data shape: {sequences.shape} ")
x_sequence=sequences[:,2:4:,:]
y_sequence=sequences[:,4,:,:]




sequences_reshaped = x_sequence.reshape(samples, -1, wspan,hspan)
sequences_reshaped=tf.constant(sequences_reshaped, dtype=tf.float32)


Y_val_onehot = tf.squeeze(tf.one_hot(tf.cast( y_sequence.reshape(-1,  wspan* hspan,1), tf.int32), num_classes))
split_index = round(samples*0.5)

x_train, x_val = sequences_reshaped[:split_index] , sequences_reshaped[split_index:]     
y_train, y_val = Y_val_onehot[:split_index], Y_val_onehot[split_index:]
split_index=len(y_val)//2
y_val, y_test = y_val[:split_index], y_val[split_index:]
x_val, x_test = x_val[:split_index] , x_val[split_index:]   



loss = lambda x, y: tf.keras.losses.categorical_crossentropy(tf.reshape(x, shape=(-1, num_classes)),
                                                             tf.reshape(y, shape=(-1, num_classes)),
                                                             from_logits=True)


shape = (wspan, hspan)
layer_dims = [10, 10,10] 


 



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
start
Training Data shape: (2100, 5, 10, 10) 


In [164]:
test_seq=sequences[0:100]
predictions2=predictions2.reshape(-1,100,2)
final=[]
errors=[]
for grid in predictions2:
    for row in grid:
     res=[-1,-1]
     res[np.argmax(row)]=1
     res[np.argmin(row)]=0
     #Only relevant if network does not acchieve 100% accuracy
     if(res[0]==-1 or res[1]==-1):
         errors.append(row)
     final.append(res)
y_test_reshaped=np.array(y_test).reshape(-1,100,2)
final_pred_reshaped=np.array(final).reshape(-1,100,2)
accuracy = (y_test_reshaped == final_pred_reshaped).mean()
print()
print(f"Test Accuracy: {accuracy} Test-Set Size: {len(x_test)}")
predictions = model.predict(x_test)


final=[]
errors=[]
for grid in predictions:
    for row in grid:
     res=[-1,-1]
     res[np.argmax(row)]=1
     res[np.argmin(row)]=0
     #Only relevant if network does not acchieve 100% accuracy
     if(res[0]==-1 or res[1]==-1):
         errors.append(row)
     final.append(res)
y_test_reshaped=np.array(y_test).reshape(-1,100,2)
final_pred_reshaped=np.array(final).reshape(-1,100,2)
accuracy = (y_test_reshaped == final_pred_reshaped).mean()
print()
print(f"Test Accuracy: {accuracy} Test-Set Size: {len(x_test)}")
print(f"Errors: {len(errors)}")


Test Accuracy: 1.0 Test-Set Size: 525
17/17 [==============================] - 0s 2ms/step

Test Accuracy: 1.0 Test-Set Size: 525
Errors: 0


In [167]:

for i in range(predictions.shape[0]):
    for y in range(predictions.shape[1]):
        for z in range(predictions.shape[2]):
            if (predictions[i][y][z] != predictions2[i][y][z]) and (np.argmax(predictions[i][y]) != np.argmax(predictions2[i][y])):
                print("Indices i:", i)
                print("Indices y:", y)
                print("Indices z:", z)


(None, 2, 100)


In [185]:
test_seq=sequences[0:100]


In [239]:
model =initialize_model_memory_debug((wspan, hspan), layer_dims, num_classes,totalistic=True,memory_horizon=MEMORY_CONSTANT) 
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=loss, metrics=['accuracy'])
 
test_seq=sequences[0:100]
predictions=[]
for x in test_seq:
    predictions.append(model.predict(np.array(x[2:4]).reshape(-1,MEMORY_CONSTANT,10,10),verbose=False))
predictions=np.array(predictions).reshape(-1,100,2)
 

y_test=[]
for v in test_seq:
    y_test.append(gol_m.mostFrequentPastStateBinaryProvided(v[2:4]))
print(len(y_test))

y_test_reshaped=np.array(y_test).reshape(-1,100,2)
final_pred_reshaped=np.array(predictions).reshape(-1,100,2)
accuracy = (y_test_reshaped == final_pred_reshaped).mean()
print()
print(f"Test Accuracy: {accuracy} Test-Set Size: {len(x_test)}")
print(f"Errors: {len(errors)}")

(None, 2, 100)
100

Test Accuracy: 1.0 Test-Set Size: 525
Errors: 0


In [220]:
predictions.shape

(5, 100, 2)

In [233]:
(model.predict(np.array(sequences[3][2:4]).reshape(-1,MEMORY_CONSTANT,10,10),verbose=False)).reshape(10,10)

array([[0., 1., 1., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 0., 1., 0., 1., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 1.],
       [1., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 1., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 1., 0., 0., 0.]], dtype=float32)

In [246]:
state=gol_m.mostFrequentPastStateBinaryProvided(sequences[0][2:4])

gol = CaMemory(grid_size=10 , rule_type=RuleTypes.OuterTotalistic,
                neighbourhood_type=CaNeighbourhoods.Von_Neumann, memory_type=MemoryTypes.Default, memory_horizon=MEMORY_CONSTANT)
gol.set_rule([[0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0, 0]])
gol.state=state
print(gol.state)
gol.step()
print(gol.state)

[[0 0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 1 1 1 0]
 [0 1 0 1 0 1 0 0 1 0]
 [0 1 0 1 0 1 1 0 1 0]
 [0 0 0 1 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 1 1 1 0]]
[[0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 0 1 1]
 [0 0 0 1 0 1 1 0 1 1]
 [0 0 1 0 0 1 1 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 0 0 0]]


In [247]:
sequences[0][4]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 1, 0, 1, 1],
       [0, 0, 1, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]])